In [1]:
from dependencies import *

STEPS

Removing links, emojies, punctuations, numbers and missing values, making them lowercase

Language detection

Tokenization

Removal of Stop Words (finding libraries for all languages)

Lemmatization

----- Preprocessing is over

LDA Topic Modeling

In [2]:
df = pd.read_csv('-1001323856649.csv')
df = df["translated"].tolist()
messages = df

In [3]:
cleaned_list = []
for x in messages:
    x = re.split('https:\/\/.*', str(x))[0] # remove links
    x = ' '.join(re.sub("[0-9.,!?:;\-='\"@#_]", " ", emoji.demojize(x)).split()) # remove punctuation, numbers, and emojies
    if x != 'nan': # check if x is not nan
        cleaned_list.append(x.lower()) # lowercase and append to cleaned_list
#cleaned_list = [x for x in cleaned_list if len(x) > 1] # remove single characters

In [25]:
cleaned_list = [x for x in cleaned_list if x not in [None, 'none', '', 'nan']]


In [5]:
# tokenize each string in the list
tokenized_list = [word_tokenize(x) for x in cleaned_list]
tokenized_list = [x for x in tokenized_list if len(x) > 1] # remove single characters

In [6]:
# remove stop words
stop_words = set(stopwords.words('english'))
for i in range(len(tokenized_list)):
    tokenized_list[i] = [x for x in tokenized_list[i] if x not in stop_words] # remove stopwords
    tokenized_list[i] = [x for x in tokenized_list[i] if len(x) > 1] # remove single characters
    tokenized_list[i] = [x for x in tokenized_list[i] if x.isalpha()] # remove non-alphabetic characters


In [7]:
tokenized_list = [x for x in tokenized_list if len(x) > 1] # remove single characters   

In [8]:
nlp = spacy.load("en_core_web_sm")
lemmatized_list = [[x for x in tokenized_list[i] if len(x) > 1] for i in range(len(tokenized_list))] # remove single characters

In [ ]:
dictionary = corpora.Dictionary(lemmatized_list)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in lemmatized_list]

# Create the LDA model
lda_model = models.LdaModel(
    corpus=doc_term_matrix,
    id2word=dictionary,
    num_topics=20,  # Define the number of topics you want to extract
    passes= 20    # Number of iterations over the corpus
)

topics = lda_model.print_topics(num_words=40)

for topic in topics:
    words = topic[1].split(" + ")
    print("Topic {}: {}".format(topic[0], ", ".join(word.split("*")[1] for word in words)))


file_name = "50_100_100"

with open(f'{file_name}.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [ ]:
# Specify the topic number you want to visualize
topic_to_visualize = 19

with open(f'50_100_100.pkl', 'rb') as f:
    lda_model = pickle.load(f)

# Iterate only once for the specified topic
for i in range(topic_to_visualize - 1, topic_to_visualize):
    words = lda_model.show_topic(i, 30)
    word_dict = {}
    for word, prob in words:
        word_dict[word] = prob
    wordcloud = WordCloud(width=800, height=400, random_state=21, max_font_size=110, background_color='white').generate_from_frequencies(word_dict)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')


    # Save the figure with a standardized filename
    plt.savefig(f'figure_{i + 1}.png')

    plt.show()



In [ ]:
#If the weight of a word is high for a particular topic, it means that the model believes that the word is strongly associated with that topic
with open('50_100_100.pkl', 'rb') as f:
    lda_model = pickle.load(f)
    
lda_model.print_topics(num_words=20)


#show without weights
topics = lda_model.print_topics(num_words=20)
for topic in topics:
    words = topic[1].split(" + ")
    print("Topic {}: {}".format(topic[0], ", ".join(word.split("*")[1] for word in words)))

